In [1]:
import numpy as np
from copy import deepcopy
from icecream import ic

In [2]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [3]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) ## valori degli oggetti
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) ## pesi degli oggetti (n.b ogni peso ha più dimensioni e i constraint devono essere rispettati in ogni dimensione)
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) ## tante quante le dimensioni 

In [4]:
CONSTRAINTS
# ogni zaino è una riga e ogni colonna è il constraint sulla dimensione

array([[ 79, 113],
       [498, 197]], dtype=int32)

## TEST PROBLEMS

In [16]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[103, 503],
       [151, 360],
       [234, 422]], dtype=int32)

In [59]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[ 374, 4644, 3513, 9344, 8385, 6376, 2446, 8174, 1909, 6013],
       [1500, 3817, 6664, 7352, 7172, 8921, 3396, 5910, 1786, 5805],
       [3446, 8040, 5006, 3314, 4818, 5875, 4008, 6116,  875, 6291],
       [6984, 7685, 7538, 7730, 3550, 7872, 9551, 4254, 4190, 8564],
       [7583, 3251, 9698, 5169, 2715, 3142, 2557, 3777, 6492, 8130],
       [8003, 6531, 1154, 3667, 6490, 1999,  688, 6678,  796, 6597],
       [ 279, 1471,  839, 4060, 1283, 6916, 9469, 1672, 6193, 7513],
       [1594, 5012, 5814, 4695, 2258, 4250, 7847, 4605, 3961, 5291],
       [4921,  610, 6408, 6507, 5738, 2033, 4165, 6232, 5650, 9276],
       [1310, 4371, 1711, 8143,   56, 6558, 8975, 6036, 1378, 6908]],
      dtype=int32)

In [6]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS


array([[96678, 31207, 42232, ..., 50847, 98523, 31153],
       [54544, 87605, 43381, ..., 46220, 96087, 40307],
       [17283, 58596, 82744, ..., 66407, 17704, 31244],
       ...,
       [34279, 12320, 49058, ..., 15014, 36235, 42651],
       [47200, 81869, 68523, ..., 86988, 40251, 23536],
       [16673, 63119, 79783, ..., 24307, 81572, 67626]],
      shape=(100, 100), dtype=int32)

SOLUTION A

In [7]:
def enforce_unique_assignment(sol):
    for i in range(NUM_ITEMS):
        assigned = np.where(sol[:, i])[0]
        if len(assigned) > 1:
            keep = np.random.choice(assigned)
            sol[:, i] = False
            sol[keep, i] = True
    return sol

In [ ]:


def tweak_non_valid_A(knapsacs:  list[set] ) -> list[set]:
    ## remove objects from all the knapsacks that are in overweight
    new_bags = deepcopy(knapsacs)
    #overweight knapsacks (bags)
    weights_per_bag = new_bags.astype(int).dot(WEIGHTS)
    overweight_mask = weights_per_bag > CONSTRAINTS
    overweight_bags = np.where(np.any(overweight_mask, axis=1))[0]

    for b in overweight_bags:
        ## IDEA: al posto di togliere uno zaino casuale possiamo togliere quello con la componente più alta che sfora
        ## excess = np.maximum(0, weights_per_bag[b] - CONSTRAINTS[b])
        items_in_bag = np.where(new_bags[b])[0]
        
        if len(items_in_bag) > 0:
            item_to_remove = np.random.choice(items_in_bag)
            new_bags[b][item_to_remove] = False 
    return new_bags

def tweak_valid_A(knapsack: list[set]) -> list[set]:
    ## it adds an item to the knapsac with the most remaining capacity
    ## if none of the left-out-items fit, it tries to swap an item with another knapsack
    new_bags = deepcopy(knapsack)
    weights_per_bag = new_bags.astype(int).dot(WEIGHTS)

    # remaining capacity per knapsack
    remaining_capacity = np.maximum(0, CONSTRAINTS - weights_per_bag)
    # knapsack with the most remaining capacity
    b = np.argmax(np.sum(remaining_capacity, axis=1))
    
    current_weight = weights_per_bag[b]
    margin = CONSTRAINTS[b] - current_weight

    # 3 groups of items:
    items_in_selected_bag = np.where(new_bags[b])[0]
    items_in_other_bags = np.where(np.any(new_bags[np.arange(NUM_KNAPSACKS) != b], axis=0))[0]
    items_left_out = np.where(~np.any(new_bags, axis=0))[0]
    

    # find items that fit in the chosen knapsack
    feasible_items = [i for i in items_left_out if np.all(WEIGHTS[i] <= margin)]
    if len(feasible_items) > 0:
        item_to_add = np.random.choice(feasible_items)
        new_bags[b][item_to_add] = True
        return new_bags
    else:
        ## swap with a random item from another knapsack
        i_remove = np.random.choice(items_in_selected_bag)
        i_add = np.random.choice(items_in_other_bags)
        knapsack_having_i_add = np.where(new_bags[:, i_add])[0][0]
        
        ## swap on the knapsack b
        new_bags[b][i_remove] = False
        new_bags[b][i_add] = True
        ## swap on the knapsack that had i_add
        
        new_bags[knapsack_having_i_add][i_add] = False
        new_bags[knapsack_having_i_add][i_remove] = True
    return new_bags

In [9]:
def cost(solution: list[set]) -> (float, float, bool):

    weights_per_bag = solution.astype(int).dot(WEIGHTS)  
    total_overweight = weights_per_bag - CONSTRAINTS
    total_value = np.sum(solution.astype(int).dot(VALUES))
    # validità: nessuna componente > 0
    valid = not np.any(total_overweight > 0)

    # penalità: somma delle eccedenze
    penalty = np.sum(np.maximum(0, total_overweight))
    # fitness: valore - penalità pesata
    alpha = 10
    fitness = total_value - alpha * penalty

    return fitness, penalty, valid

In [17]:
#random solution
current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True
first_solution = deepcopy(current_solution)
first_solution


array([[ True,  True, False, False, False, False, False, False,  True,
        False,  True, False, False, False, False, False, False, False,
        False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False,  True, False, False, False, False, False,  True,
        False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
         True, False]])

In [24]:


(current_fitness, current_cost, current_valid) = cost(current_solution)

MAX_STEPS = 500


for steps in range(MAX_STEPS):

    ic(steps, current_cost, current_fitness, current_valid)
    if current_valid:
        ## valid solution
        new_solution = tweak_valid_A(current_solution)
        new_solution = enforce_unique_assignment(new_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)
        if new_valid:
            if new_fitness>= current_fitness: 
                ## la nuova soluzione è valida e il costo si avvicina a zero sempre di più (costo ideale)
                current_cost= new_cost
                current_fitness= new_fitness
                current_solution = new_solution
                current_valid= new_valid
                continue

    else: 
        ## invalid solution
        new_solution = tweak_non_valid_A(current_solution)
        new_solution = enforce_unique_assignment(new_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)

        if not new_valid: 
            
        ## la nuova soluzione non è valida anche essa 
            if new_cost < current_cost or (new_cost== current_cost and new_fitness>=current_fitness): 
                # la nuova soluzione si avvicina a essere valida 
                current_cost = new_cost
                current_solution = new_solution
                current_fitness= new_fitness
                current_valid= new_valid
                continue
            else:
                continue
        else:
            ## la nuova soluzione è valida 
            current_cost = new_cost
            current_solution = new_solution
            current_fitness= new_fitness
            current_valid= new_valid
            continue

    

print(current_cost, current_fitness, current_valid)



ic| steps: 0
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 1
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 2
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 3
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 4
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 5
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 6
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 7
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 8
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    current_valid: True
ic| steps: 9
    current_cost: np.int64(0)
    current_fitness: np.int64(374)
    

0 374 True


In [19]:
solution=current_solution
solution

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False,  True, False, False, False, False, False,  True,
        False, False],
       [False, False, False, False,  True, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
         True, False]])

In [20]:
# Check if the solution is valid
valid_per_bag = []
for b in range(NUM_KNAPSACKS):
    # seleziona gli item presenti nello zaino b
    items_in_b = np.where(solution[b])[0]            # indici degli item presenti
    total_weight = WEIGHTS[items_in_b].sum(axis=0)   # somma per dimensione
    valid_per_bag.append(np.all(total_weight <= CONSTRAINTS[b]))

overall_valid = all(valid_per_bag)
overall_valid

True

In [21]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.True_

In [22]:
def solution_evaluation(solution):
    total_value = 0
    percentage_weight_over_constraint = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]
        total_value += knapsack_value
        percentage_weight_over_constraint[b] = knapsack_weight*100/CONSTRAINTS[b]


    return percentage_weight_over_constraint, total_value

solution_evaluation(solution)

(array([[94.17475728,  6.56063618],
        [73.50993377, 48.05555556],
        [56.41025641, 24.40758294]]),
 np.int32(374))